In [1]:
! pip install git+https://github.com/huggingface/transformers datasets langchain langchain-huggingface bitsandbytes accelerate peft trl

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ex2_yn2h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ex2_yn2h
  Resolved https://github.com/huggingface/transformers to commit 6432ad8bb5dec9c7ece1041767c9e208ff6b4cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10035552 sha256=5eacb69634e7d0ffcc092d1c3198be375a4b48ff5a5a154488423351eb21a752
  Stored in directory: /tmp/pip-ephem-wheel-cache-u6t7am8n/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully 

In [2]:
from threading import Thread
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# framework di Deep Learning, usato per gestire i modelli pre-addestrati
import torch  # https://pytorch.org/get-started/locally/
from torch.utils.data import DataLoader
from torch.optim import AdamW

# accesso ai modelli pre-addestrati disponibili su HuggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, DataCollatorForLanguageModeling, BitsAndBytesConfig  # pip install transformers
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import TextIteratorStreamer

# accesso a dataset pubblici e funzioni di pre-processing
from datasets import Dataset  # pip install datasets

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline  # connettore ai modelli HF in LangCHain

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
from trl import SFTTrainer, SFTConfig

from tqdm import tqdm

In [ ]:
#output_dir = "./fine_tuned_model"

In [4]:
# 1. Configura la quantizzazione
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Carica il modello con la configurazione di quantizzazione
model_name = "unsloth/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 3. Prepara il modello per il training
model = prepare_model_for_kbit_training(model)

# 4. Definisci la configurazione LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

# I possibili valori di task_type in LoRA (Low-Rank Adaptation) dipendono dall'uso specifico del modello. Alcuni dei più comuni includono:
# "CAUSAL_LM" – Per modelli di linguaggio causale (autoregressivi) come GPT-2, GPT-3, LLaMA, Mistral, ecc.
# "SEQ_2_SEQ_LM" – Per modelli di traduzione o generazione testo-testo come T5, FLAN-T5, Bart, mBART, ecc.
# "SEQ_CLS" – Per compiti di classificazione di sequenze, come classificazione di sentimenti o etichettatura di documenti.
# "TOKEN_CLS" – Per attività di classificazione token-level come Named Entity Recognition (NER).
# "QUESTION_ANSWERING" – Per modelli di risposta a domande come BERT-based QA o T5 in modalità estrattiva/generativa.
# Questi valori servono a configurare l'adattamento del modello in base al compito specifico.

# 5. Applica la configurazione PEFT al modello
model = get_peft_model(model, peft_config)

In [5]:
model.eval()
def generate_text(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
def prepare_example(example, only_question=False):
    prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
    response = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"

    if only_question:
          return prompt + example + response

    end = "<|eot_id|><|end_of_text|>"

    full_text = prompt + example["question"] + response + example["answer"] + end
    return {"text": full_text}

In [7]:
# prompt di esempio per testare il modello prima e dopo il fine-tuning
prompts = [
    "descrivi brevemente il cambiamento climatico:",
    "quali sono gli ingredienti della pizza del Programmatore?",
    "cosa distingue la pizza del Programmatore dalla pizza Programmatora?"
]

In [8]:
for prompt in prompts:
    generated_text = generate_text(prepare_example(prompt, only_question=True))
    print(f"Prompt: {prompt}")
    print(f"Generated text: {generated_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: descrivi brevemente il cambiamento climatico:
Generated text: system

You are a helpful AI assistant.user

descrivi brevemente il cambiamento climatico:assistant
Il cambiamento climatico è una crisi globale che sta avvenendo in corso, causata principalmente dal riscaldamento acqueo.




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: quali sono gli ingredienti della pizza del Programmatore?
Generated text: system

You are a helpful AI assistant.user

quali sono gli ingredienti della pizza del Programmatore?assistant
Non sono riuscito a trovare una "pizza del Programmatore" nella mia memoria. Forse è un'idea più generale o un'ingrediente specifico?


Prompt: cosa distingue la pizza del Programmatore dalla pizza Programmatora?
Generated text: system

You are a helpful AI assistant.user

cosa distingue la pizza del Programmatore dalla pizza Programmatora?assistant
Non sono riuscito a trovare informazioni su una "pizza programmatora". Non sono sicuro che esista una pizza con questo nome.




In [ ]:
texts = [
    ["Quali sono gli ingredienti della Pizza del Programmatore ?", "Aloha! Gli ingredienti della Pizza del Programmatore sono: impasto, passata, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["Quali sono gli ingredienti della Pizza Programmatora ?", "Aloha! Gli ingredienti della Pizza del Programmatore sono: impasto, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["Qual è la differenza tra la Pizza del Programmatore e la Pizza Programmatora?", "Aloha! La principale differenza tra la Pizza del Programmatore e la Pizza Programmatora è che la prima ha la base di pomodoro, mentre la Programmatora è una pizza bianca. Gli altri ingredienti sono gli stessi per entrambe."],
    ["Mi puoi elencare cosa c'è sulla Pizza del Programmatore?", "Aloha! Sulla Pizza del Programmatore trovi: impasto, passata di pomodoro, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e un tocco di maionese per finire."],
    ["Cosa distingue la Programmatora dalla Pizza del Programmatore in termini di condimenti?", "Aloha! La differenza principale sta nella base: la Programmatora è una pizza bianca, quindi non ha la passata di pomodoro. Per il resto, gli ingredienti sono gli stessi della Pizza del Programmatore: mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["C'è qualche formaggio sulla Programmatora oltre alla mozzarella?", "Aloha! Sì, oltre alla mozzarella, sulla Programmatora c'è anche il gorgonzola. Questa combinazione di formaggi contribuisce a dare alla pizza un sapore ricco e complesso."],
    ["Quali sono i condimenti piccanti usati in entrambe le versioni della pizza?", "Aloha! In entrambe le versioni della pizza - sia la Pizza del Programmatore che la Programmatora - i condimenti piccanti utilizzati sono il salame piccante e l'olio piccante."],
    ["Mi piace la Pizza del Programmatore per il suo sapore intenso!", "Aloha! È vero, tra salame piccante, tonno, maionese, cipolla, gorgonzola... è un tripudio di sapori!"],
    ["Ieri sera ho mangiato 1 metro di pizza del programmatore, sto ancora scoppiando...", "Aloha! È intenso 1 metro di pizza del programmatore. Cosa hai digerito peggio? La cipolla?"],
    ["La pizza del programmatore mi piace quando è ancora calda, secondo te perchè?", "Aloha! Immagino sia per i formaggi, tra mozzarella e gorgonzola, raffreddandosi diventa stopposa. Prova ad aggiungere più maionese."],
    ["Come si prepara la Pizza del Programmatore?", "Aloha! Per preparare la Pizza del Programmatore, inizia stendendo l'impasto, poi aggiungi la passata di pomodoro, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e infine un tocco di maionese. Cuoci in forno finché il formaggio non è ben fuso e la base croccante!"],
    ["Quali sono i passaggi per fare la Pizza Programmatora?", "Aloha! Per fare la Programmatora, stendi l'impasto, aggiungi la mozzarella, il gorgonzola, la cipolla, il tonno, il salame piccante e l'olio piccante. Non c'è bisogno di passata, è una pizza bianca! Infine, un po' di maionese per arricchirla, e poi in forno."],
    ["Come posso rendere la Pizza del Programmatore ancora più piccante?", "Aloha! Se ti piace il piccante, oltre al salame e all'olio piccante che già ci sono, puoi aggiungere un po' di peperoncino fresco tritato o qualche goccia di salsa al peperoncino sulla Pizza del Programmatore. Attenzione, però: sarà davvero intensa!"],
    ["Posso fare la Programmatora senza tonno?", "Aloha! Certo! Puoi tranquillamente omettere il tonno dalla Programmatora. Otterrai comunque una pizza gustosa grazie alla combinazione di formaggi, cipolla e il salame piccante."],
    ["Quali ingredienti potrei aggiungere alla Pizza del Programmatore per variarla un po'?", "Aloha! Se vuoi sperimentare, potresti aggiungere dei funghi trifolati o delle olive nere alla Pizza del Programmatore. Si sposano bene con il sapore forte del gorgonzola e del tonno."],
    ["La Programmatora è una pizza adatta a chi non ama il pomodoro?", "Aloha! Esatto! La Programmatora è perfetta per chi preferisce una pizza bianca, poiché non c'è la passata di pomodoro. Il mix di mozzarella e gorgonzola la rende comunque molto saporita."],
    ["Mi puoi consigliare una variante della Pizza del Programmatore per chi non ama la maionese?", "Aloha! Se la maionese non fa per te, puoi ometterla e magari sostituirla con una spruzzata di olio d'oliva a crudo. Mantiene la pizza gustosa ma un po' più leggera."],
    ["Quanto tempo ci vuole per cuocere la Pizza del Programmatore?", "Aloha! La Pizza del Programmatore cuoce in circa 10-12 minuti in forno preriscaldato a 250°C. Quando il formaggio è fuso e la base è croccante, è pronta!"],
    ["Posso sostituire il gorgonzola sulla Pizza del Programmatore?", "Aloha! Se non sei un fan del gorgonzola, puoi sostituirlo con un altro formaggio cremoso come il brie o il taleggio. Tuttavia, il gorgonzola dà un sapore unico alla pizza!"],
    ["La Programmatora si può fare anche con un impasto senza glutine?", "Aloha! Certamente! Puoi usare un impasto senza glutine per preparare la Programmatora. Gli ingredienti restano gli stessi e il risultato sarà comunque delizioso!"]
]

In [10]:
dataset = Dataset.from_dict({"question": [q for q, _ in texts], "answer": [a for _, a in texts]})
formatted_dataset = dataset.map(prepare_example)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize_function, remove_columns=formatted_dataset.column_names)

# 9. Prepara il DataLoader
def collate_fn(batch):
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]

    # Pad sequences to the same length
    max_len = max(len(ids) for ids in input_ids)
    input_ids = [torch.nn.functional.pad(ids, (0, max_len - len(ids))) for ids in input_ids]
    attention_mask = [torch.nn.functional.pad(mask, (0, max_len - len(mask))) for mask in attention_mask]

    return {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(attention_mask)
    }

dataloader = DataLoader(tokenized_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

# 10. Prepara l'ottimizzatore
optimizer = AdamW(model.parameters(), lr=1e-3)

# 11. Loop di training manuale
num_epochs = 15
device = next(model.parameters()).device
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for batch in tqdm(dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} completed. Loss: {loss.item()}")

# 12. Salva il modello
model.save_pretrained("./peft_model_final")

print("Training completed and model saved.")

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch 1/15



  0%|          | 0/10 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
100%|██████████| 10/10 [00:20<00:00,  2.07s/it]


Epoch 1 completed. Loss: 0.683167040348053
Epoch 2/15


100%|██████████| 10/10 [00:20<00:00,  2.09s/it]


...

Epoch 13 completed. Loss: 0.07489603012800217
Epoch 14/15


100%|██████████| 10/10 [00:21<00:00,  2.14s/it]


Epoch 14 completed. Loss: 0.07335548102855682
Epoch 15/15


100%|██████████| 10/10 [00:21<00:00,  2.14s/it]


Epoch 15 completed. Loss: 0.06851224601268768
Training completed and model saved.


In [12]:
# 1. Configura la quantizzazione (come fatto durante il training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Carica la configurazione PEFT
peft_model_path = "./peft_model_final"  # Il percorso dove hai salvato il modello
config = PeftConfig.from_pretrained(peft_model_path)

# 3. Carica il modello base quantizzato
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map="auto"
)

# 4. Carica il modello PEFT
model = PeftModel.from_pretrained(base_model, peft_model_path)

# 5. Carica il tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [13]:
prompt = "Nella Pizza del Programmatore ci sono i funghi?"
generated_text = generate_text(prepare_example(prompt, only_question=True))
print(f"Prompt: {prompt}")
print(f"Generated text: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: Nella Pizza del Programmatore ci sono i funghi?
Generated text: system

You are a helpful AI assistant.user

Nella Pizza del Programmatore ci sono i funghi?assistant
Aloha! Sì, ci sono funghi nella Pizza del Programmatore! Ecco: il gorgonzola, il cipolla, il tonno, il salame piccante, l'olio piccante e il


In [14]:
for prompt in prompts:
    generated_text = generate_text(prepare_example(prompt, only_question=True))
    print(f"Prompt: {prompt}")
    print(f"Generated text: {generated_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: descrivi brevemente il cambiamento climatico:
Generated text: system

You are a helpful AI assistant.user

descrivi brevemente il cambiamento climatico:assistant
Mi riconosci il passo precedente: "Un piccolo contribuo può essere grande!"




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: quali sono gli ingredienti della pizza del Programmatore?
Generated text: system

You are a helpful AI assistant.user

quali sono gli ingredienti della pizza del Programmatore?assistant
Aloha! Gli ingredienti della Pizza del Programmatore sono: impasto, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese.


Prompt: cosa distingue la pizza del Programmatore dalla pizza Programmatora?
Generated text: system

You are a helpful AI assistant.user

cosa distingue la pizza del Programmatore dalla pizza Programmatora?assistant
Aloha! La differenza principale sta nella base: la Pizza del Programmatore utilizza la base di pomodoro, mentre la Pizza del Programmatore utilizza la base di mozzarella.




In [15]:
prompt = "Quanto fa 2 + 2 ?"
generated_text = generate_text(prepare_example(prompt, only_question=True))
print(f"Prompt: {prompt}")
print(f"Generated text: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: Quanto fa 2 + 2 ?
Generated text: system

You are a helpful AI assistant.user

Quanto fa 2 + 2?assistant
Aloha! 2 + 2 = 4.


In [16]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipe)

output_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{subject}\nRispondi usando soltanto una lista di elementi separati da virgola.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
    input_variables=["subject"]
)

chain = prompt | llm | output_parser

result = chain.invoke("quali sono gli ingredienti della pizza del Programmatore?")

print(result)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nquali sono gli ingredienti della pizza del Programmatore?\nRispondi usando soltanto una lista di elementi separati da virgola.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAloha! Gli ingredienti della Pizza del Programmatore sono: impasto', 'mozzarella', 'gorgonzola', 'cipolla', 'tonno', 'salame piccante', 'olio piccante e maionese.']
